In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.MachineModels import MachineModels 
from src.predictionModule.FilterSamples import FilterSamples

import numpy as np
import datetime
import pandas as pd
import polars as pl
import optuna
import scipy

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-lstm-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-09-02 09:59:49,528 - This will print to the notebook's output cell


c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
params_default = {
    "idxAfterPrediction": 5,
    'timesteps': 90,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 2,
    "LSTM_dropout": 0.001,
    "LSTM_recurrent_dropout": 0.001,
    "LSTM_learning_rate": 0.001,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 10,
    "LSTM_l1": 0.001,
    "LSTM_l2": 0.001,
    "LSTM_inter_dropout": 0.001,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",
}

In [3]:
stock_group = "group_regOHLCV_over5years"
treegroup = "group_finanTo2011"
eval_date = datetime.date(year=2025, month=7, day=13)
split_date = datetime.date(year=2025, month=1, day=1)

studytime = 60*60*1
studyname = f"sandbox_lstm_optuna_{formatted_str}"
n_startup_trials = 7
q_full = 0.986

In [ ]:
def objective(trial: optuna.Trial) -> float:
    opt_params = params_default.copy()
    opt_params["year_start"] = trial.suggest_int("year_start", 2019, 2021)
    opt_params["idxAfterPrediction"] = 5
    opt_params["LoadupSamples_time_inc_factor"] = trial.suggest_int("LoadupSamples_time_inc_factor", 21, 81, step=5)
    opt_params["timesteps"] = trial.suggest_int("timesteps", 30, 90, step=5)
    opt_params["LSTM_units"] = 32
    opt_params["LSTM_num_layers"] = trial.suggest_int("LSTM_num_layers", 1, 2)
    opt_params["LSTM_learning_rate"] = trial.suggest_float("LSTM_learning_rate", 1e-5, 1e-3, log=True)
    opt_params["LSTM_epochs"] = 2
    opt_params["LSTM_l1"] = trial.suggest_float("LSTM_l1", 1e-5, 1e-1, log=True)
    opt_params["LSTM_l2"] = trial.suggest_float("LSTM_l2", 1e-4, 5e-1, log=True)
    opt_params["LSTM_dropout"] = trial.suggest_float("LSTM_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_inter_dropout"] = trial.suggest_float("LSTM_inter_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_recurrent_dropout"] = trial.suggest_float("LSTM_recurrent_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_conv1d_kernel_size"] = 3
    opt_params["is_single_feature"] = trial.suggest_categorical("is_single_feature", [False, True])

    ls = LoadupSamples(
        train_start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        test_dates=[eval_date],
        treegroup=treegroup,
        timegroup=stock_group,
        params=opt_params,
    )
    ls.load_samples(main_path = "../src/featureAlchemy/bin/")

    ls.split_dataset(
        start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        last_train_date=split_date,
        last_test_date=eval_date,
    )
    fs_pre = FilterSamples(
        Xtree_train=ls.train_Xtree,
        ytree_train=ls.train_ytree,
        treenames=ls.featureTreeNames,
        Xtree_test=ls.test_Xtree,
        ytree_test=ls.test_ytree,
        meta_train=ls.meta_pl_train,
        meta_test=ls.meta_pl_test,
        params=opt_params
    )

    cat_mask_train, cat_mask_test = fs_pre.categorical_masks()
    ls.apply_masks(cat_mask_train, cat_mask_test)

    Xtrain = ls.train_Xtime
    ytrain = ls.train_ytime
    Xtest = ls.test_Xtime
    ytest = ls.test_ytime

    true_res = ls.meta_pl_test

    Xtrain = Xtrain[:, -opt_params["timesteps"]:, :]
    Xtest = Xtest[:, -opt_params["timesteps"]:, :]

    if opt_params["is_single_feature"]:
        Xtrain = Xtrain[:, :, [0]]
        Xtest = Xtest[:, :, [0]]

    mm = MachineModels(opt_params)

    starttime0 = datetime.datetime.now()
    model_lstm0, res_dict0 = mm.run_LSTM_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
    preds_test0 = mm.predict_LSTM_torch(model_lstm0, Xtest, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    endtime0 = datetime.datetime.now()

    # final mask
    q0 = q_full
    mask_pred0_test_above = (preds_test0 >= np.quantile(preds_test0,   q0))
    mask_pred0_test_below = (preds_test0 <= np.quantile(preds_test0, 1-q0))

    true_res_masked_above = true_res.filter(pl.Series(mask_pred0_test_above))
    true_res_masked_below = true_res.filter(pl.Series(mask_pred0_test_below))

    score = (scipy.stats.gmean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/opt_params["idxAfterPrediction"])

    # Log some results
    def quant_dis_in_mask(mask: np.ndarray, q: float) -> float:
        if not mask.any():
            return len(mask)
        return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)

    fullmask_above = mask_pred0_test_above.copy()
    fullmask_below = mask_pred0_test_below.copy()

    logger.info(f"Trial {trial.number} with params: {opt_params}")
    logger.info(f"  q0: {q0:.4f}")
    logger.info(f"  Duration0: {endtime0 - starttime0}")
    logger.info(f"  Val RMSE0 adjusted: {res_dict0['val_rmse']/opt_params['LoadupSamples_time_inc_factor']:.4f}")
    logger.info(f"  Mean all prediction: {scipy.stats.gmean(true_res['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean above prediction: {scipy.stats.gmean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean below prediction: {scipy.stats.gmean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Quantile 0.99 for distance in mask above: {quant_dis_in_mask(fullmask_above, 0.99)}")
    logger.info(f"  Quantile 0.99 for distance in mask below: {quant_dis_in_mask(fullmask_below, 0.99)}")
    logger.info(f"  Ratio for quantile-distance-to-length above: {quant_dis_in_mask(fullmask_above, 0.99) / len(fullmask_above):.4f}")
    logger.info(f"  Ratio for quantile-distance-to-length below: {quant_dis_in_mask(fullmask_below, 0.99) / len(fullmask_below):.4f}")
    logger.info(f"  Score: {score:.4f}")

    return score

optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=n_startup_trials)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(objective, timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-09-02 09:59:51,301] A new study created in RDB with name: sandbox_lstm_optuna_notebook-lstm-optuna-02sep25_0959


2025-09-02 09:59:51,301 - A new study created in RDB with name: sandbox_lstm_optuna_notebook-lstm-optuna-02sep25_0959
2025-09-02 10:00:48,051 - Test date 2025-07-13 not found in the database. Omitting.
2025-09-02 10:00:55,166 - Non-finite/too-large values in train y tree: 15 samples.
2025-09-02 10:00:55,166 - Non-finite/too-large values in train X tree: 121 samples.
2025-09-02 10:00:55,169 - Non-finite/too-large values in train y time: 15 samples.
2025-09-02 10:00:55,175 - Removing 136 samples from training data.


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

2025-09-02 10:01:43,372 - Epoch 1/2 — Train RMSE: 0.5317 — Validation RMSE: 0.4230


Epochs:  50%|█████     | 1/2 [00:14<00:14, 14.32s/it]

...

2025-09-02 10:26:14,531 - Epoch 2/2 — Train RMSE: 0.3896 — Validation RMSE: 0.3919


Epochs: 100%|██████████| 2/2 [00:29<00:00, 14.84s/it]


2025-09-02 10:26:15,162 - Trial 12 with params: {'idxAfterPrediction': 5, 'timesteps': 75, 'target_option': 'last', 'LoadupSamples_time_scaling_stretch': True, 'LoadupSamples_time_inc_factor': 51, 'LSTM_units': 32, 'LSTM_num_layers': 1, 'LSTM_dropout': 0.016457730033522917, 'LSTM_recurrent_dropout': 0.0001025728153449527, 'LSTM_learning_rate': 0.00024580712009282415, 'LSTM_optimizer': 'adam', 'LSTM_bidirectional': True, 'LSTM_batch_size': 4096, 'LSTM_epochs': 2, 'LSTM_l1': 0.0004140892352149675, 'LSTM_l2': 0.009484663015153223, 'LSTM_inter_dropout': 0.004134708556625846, 'LSTM_input_gaussian_noise': 0.001, 'LSTM_conv1d': True, 'LSTM_conv1d_kernel_size': 3, 'LSTM_loss': 'mse', 'year_start': 2019, 'is_single_feature': False}
2025-09-02 10:26:15,163 -   q0: 0.9860
2025-09-02 10:26:15,164 -   Duration0: 0:00:30.371964
2025-09-02 10:26:15,164 -   Val RMSE0 adjusted: 0.0077
2025-09-02 10:26:15,164 -   Mean all prediction: 1.0003
2025-09-02 10:26:15,164 -   Mean above prediction: 1.0045
2025-

[I 2025-09-02 10:26:15,691] Trial 12 finished with value: 1.0008946860211465 and parameters: {'year_start': 2019, 'LoadupSamples_time_inc_factor': 51, 'timesteps': 75, 'LSTM_num_layers': 1, 'LSTM_learning_rate': 0.00024580712009282415, 'LSTM_l1': 0.0004140892352149675, 'LSTM_l2': 0.009484663015153223, 'LSTM_dropout': 0.016457730033522917, 'LSTM_inter_dropout': 0.004134708556625846, 'LSTM_recurrent_dropout': 0.0001025728153449527, 'is_single_feature': False}. Best is trial 3 with value: 1.0038045605340327.


2025-09-02 10:26:15,691 - Trial 12 finished with value: 1.0008946860211465 and parameters: {'year_start': 2019, 'LoadupSamples_time_inc_factor': 51, 'timesteps': 75, 'LSTM_num_layers': 1, 'LSTM_learning_rate': 0.00024580712009282415, 'LSTM_l1': 0.0004140892352149675, 'LSTM_l2': 0.009484663015153223, 'LSTM_dropout': 0.016457730033522917, 'LSTM_inter_dropout': 0.004134708556625846, 'LSTM_recurrent_dropout': 0.0001025728153449527, 'is_single_feature': False}. Best is trial 3 with value: 1.0038045605340327.
2025-09-02 10:27:15,424 - Test date 2025-07-13 not found in the database. Omitting.
2025-09-02 10:27:22,472 - Non-finite/too-large values in train y tree: 15 samples.
2025-09-02 10:27:22,475 - Non-finite/too-large values in train X tree: 121 samples.
2025-09-02 10:27:22,479 - Non-finite/too-large values in train y time: 15 samples.
2025-09-02 10:27:22,481 - Removing 136 samples from training data.


In [ ]:
df.to_parquet(f"{formatted_str}.parquet", index=False)